In [18]:
import pandas as pd

In [19]:
num=1000
pd.read_csv(f'https://www.historique-meteo.net/site/export.php?ville_id={num}')

HTTPError: HTTP Error 403: Forbidden

In [21]:
from selenium import webdriver
import chromedriver_binary

from selenium.webdriver.common.by import By
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as ec

In [69]:

import zipfile

from selenium import webdriver



PROXY_HOST = '154.95.0.22'  # rotating proxy or host
PROXY_PORT = 6275 # port
PROXY_USER = 'xatdbibj' # username
PROXY_PASS = 'y0kzxgfiz3ne' # password


manifest_json = """
{
    "version": "1.0.0",
    "manifest_version": 2,
    "name": "Chrome Proxy",
    "permissions": [
        "proxy",
        "tabs",
        "unlimitedStorage",
        "storage",
        "<all_urls>",
        "webRequest",
        "webRequestBlocking"
    ],
    "background": {
        "scripts": ["background.js"]
    },
    "minimum_chrome_version":"22.0.0"
}
"""

background_js = """
var config = {
        mode: "fixed_servers",
        rules: {
        singleProxy: {
            scheme: "http",
            host: "%s",
            port: parseInt(%s)
        },
        bypassList: ["localhost"]
        }
    };

chrome.proxy.settings.set({value: config, scope: "regular"}, function() {});

function callbackFn(details) {
    return {
        authCredentials: {
            username: "%s",
            password: "%s"
        }
    };
}

chrome.webRequest.onAuthRequired.addListener(
            callbackFn,
            {urls: ["<all_urls>"]},
            ['blocking']
);
""" % (PROXY_HOST, PROXY_PORT, PROXY_USER, PROXY_PASS)


def get_chromedriver(use_proxy=False, user_agent=None):
    chrome_options = webdriver.ChromeOptions()
    if use_proxy:
        pluginfile = 'proxy_auth_plugin.zip'

        with zipfile.ZipFile(pluginfile, 'w') as zp:
            zp.writestr("manifest.json", manifest_json)
            zp.writestr("background.js", background_js)
        chrome_options.add_extension(pluginfile)
    if user_agent:
        chrome_options.add_argument('--user-agent=%s' % user_agent)
    driver = webdriver.Chrome(
        chrome_options=chrome_options)
    return driver



def get_ville_id(driver,dep,ville):
    #driver=webdriver.Chrome()
    driver.get(f'https://www.historique-meteo.net/france/{dep}/{ville}/')

    #driver.find_element(By.CLASS_NAME,"form-control").clear()
    #driver.find_element(By.CLASS_NAME,"form-control").send_keys("dijon")
    #driver.find_element(By.CLASS_NAME,"btn.btn-theme.btn-lg").click()
    
    try:
        page=driver.find_element(By.CLASS_NAME,"btn.btn-small.btn-theme.pull-right")
        lien=page.get_attribute("href")
        start=lien.index('=')
        return(lien[start+1:])
    except:
        return 'erreur 404'

In [71]:
dict_id_ville={}
driver = get_chromedriver(use_proxy=True)
for i,j in zip(liste_regions,liste_ville):
    id =get_ville_id(driver,i,j)
    print(i,j,id)
    dict_id_ville[j]=id
driver.close()

/var/folders/v0/5qktxwz92kl52b34cz5nxqbr0000gn/T/ipykernel_86400/3461726541.py:77: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(


picardie amiens 226
champagne-ardenne reims 177
nord-pas-de-calais lille 210
normandie rouen 218
champagne-ardenne troyes 178
poitou-charentes poitiers 233
centre tours 168
bretagne rennes 172
bourgogne dijon 170
bretagne lannion 1002
normandie caen 216
centre bourges 163
normandie alencon-orne 873
charentes saintes-charente-maritime 5655
bretagne vannes 173
lorraine nancy 200
bretagne concarneau 1043
pays-de-la-loire nantes 221
normandie granville 2181


In [70]:
liste_regions = ['picardie', 'champagne-ardenne', 'nord-pas-de-calais', 'normandie',
                 'champagne-ardenne', 'poitou-charentes', 'centre', 'bretagne', 'bourgogne',
                 'bretagne', 'normandie', 'centre', 'normandie', 'charentes', 'bretagne',
                 'lorraine', 'bretagne', 'pays-de-la-loire', 'normandie']

liste_ville = ['amiens', 'reims', 'lille', 'rouen', 'troyes', 'poitiers', 'tours',
               'rennes', 'dijon', 'lannion', 'caen', 'bourges', 'alencon-orne', 'saintes-charente-maritime',
               'vannes', 'nancy', 'concarneau', 'nantes', 'granville']

In [72]:
dict_id_ville

{'amiens': '226',
 'reims': '177',
 'lille': '210',
 'rouen': '218',
 'troyes': '178',
 'poitiers': '233',
 'tours': '168',
 'rennes': '172',
 'dijon': '170',
 'lannion': '1002',
 'caen': '216',
 'bourges': '163',
 'alencon-orne': '873',
 'saintes-charente-maritime': '5655',
 'vannes': '173',
 'nancy': '200',
 'concarneau': '1043',
 'nantes': '221',
 'granville': '2181'}